In [ ]:
import socceraction.spadl as spadl
import socceraction.xthreat as xthreat
import soccerdata as sd
from pathlib import PosixPath
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
xTModell = xthreat.load_model(
    "/home/morten/Develop/packing-report/xT-impact/models/xT_test"
)

In [ ]:
choosen_match_id = 1557933

In [ ]:
ws = sd.WhoScored(
    leagues="GER-Bundesliga2",
    seasons=[21],
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
    path_to_browser="/usr/bin/chromium",
    headless=True,
)

loader = ws.read_events(match_id=choosen_match_id, output_fmt="loader")

In [ ]:
df_teams = loader.teams(game_id=choosen_match_id)
df_teams

In [ ]:
df_players = loader.players(game_id=choosen_match_id)
df_players

In [ ]:
df_events = loader.events(game_id=choosen_match_id)
display(df_events.head())

In [ ]:
spadl.results_df()

In [ ]:
actions = {}
actions[df_events["game_id"][0]] = spadl.opta.convert_to_actions(
    df_events, df_teams["team_id"][0]
)
# 2. Convert direction of play
actions_ltr = pd.concat(
    [spadl.play_left_to_right(actions[df_events["game_id"][0]], df_teams["team_id"][0])]
)

shoot_actions = actions_ltr[actions_ltr["type_id"] == 11]
print(shoot_actions.shape)

In [ ]:
xG_rating = xTModell.rate_xG(shoot_actions)
player_values = {}
for y in shoot_actions["player_id"]:
    player_values[y] = 0
for indx, y in enumerate(shoot_actions["player_id"]):
    player_values[y] += xG_rating[indx]

In [ ]:
for x in player_values.keys():
    print(
        f"{x} : {df_players[df_players['player_id'] == x]['player_name'].values[0]} = {player_values[x]}"
    )

In [ ]:
team_values = {}
for x in df_teams["team_id"]:
    team_values[x] = 0
for x in player_values.keys():
    team_values[
        df_players[df_players["player_id"] == x]["team_id"].values[0]
    ] += player_values[x]

team_values

In [ ]:
player_vals, pl_id, tm_id, xt_im = {}, [], [], []
for x in player_values.keys():
    pl_id.append(x)
    tm_id.append(df_players[df_players["player_id"] == x]["player_name"].values[0])
    xt_im.append(
        player_values[x]
        * (
            player_values[x]
            / team_values[df_players[df_players["player_id"] == x]["team_id"].values[0]]
        )
    )

player_vals = {"player_id": pl_id, "name": tm_id, "xG_impact": xt_im}
impact_df = pd.DataFrame.from_dict(player_vals)

In [ ]:
pd.options.display.float_format = "{:.10f}".format
impact_df.sort_values("xG_impact", ascending=False)

In [ ]:
ws = sd.WhoScored(
    leagues="GER-Bundesliga2",
    seasons=[21],
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
    path_to_browser="/usr/bin/chromium",
    headless=True,
)

loader = ws.read_events(output_fmt="loader")

In [ ]:
df_games = loader.games(competition_id="GER-Bundesliga2", season_id="2122")

In [ ]:
ce = sd.ClubElo(
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
)

In [ ]:
# succ_actions_dict
def shot_actions(game_id, df_events, df_games):
    actions = {}
    actions[game_id] = spadl.opta.convert_to_actions(
        df_events, df_games[df_games["game_id"] == game_id].home_team_id.values[0]
    )
    # 2. Convert direction of play
    actions_ltr = pd.concat(
        [
            spadl.play_left_to_right(
                actions[game_id],
                df_games[df_games["game_id"] == game_id].home_team_id.values[0],
            )
        ]
    )
    shot_actions_df = actions_ltr[actions_ltr["type_id"] == 11]
    return shot_actions_df


def fill_ids(succs_events, df_players):
    for y in succs_events["player_id"]:
        if y not in id_to_name:
            player_name = df_players[df_players["player_id"] == y][
                "player_name"
            ].values[0]
            id_to_name[y] = df_players[df_players["player_id"] == y][
                "player_name"
            ].values[0]
            id_to_xt_impact[y] = 0
            id_to_xt_impact_elo[y] = 0


def reset_player_xt_values(succs_events):
    player_values = {}
    for indx, y in enumerate(succs_events["player_id"]):
        player_values[y] = 0
    return player_values


def get_club_elo_factorial(date, team, league):
    print("lol")

In [ ]:
id_to_name, id_to_xt_impact, id_to_xt_impact_elo = {}, {}, {}

for idx, game_id in enumerate(tqdm(df_games["game_id"])):
    df_events = loader.events(game_id=game_id)
    shot_actions_df = shot_actions(game_id, df_events, df_games)
    xt_ratings = xTModell.rate_xG(shot_actions_df)
    df_players = loader.players(game_id=game_id)
    # fill_ids(succs_events, df_players)
    for y in shot_actions_df["player_id"]:
        if y not in id_to_name:
            player_name = df_players[df_players["player_id"] == y][
                "player_name"
            ].values[0]
            id_to_name[y] = df_players[df_players["player_id"] == y][
                "player_name"
            ].values[0]
            id_to_xt_impact[y] = 0
            id_to_xt_impact_elo[y] = [[], [], []]
    player_values = reset_player_xt_values(shot_actions_df)
    for indx, y in enumerate(shot_actions_df["player_id"]):
        player_values[y] += xt_ratings[indx]
    for x in player_values.keys():
        if player_values[x] < 0:
            player_values[x] = 0
    team_values = {}
    df_teams = loader.teams(game_id=game_id)
    ## reset team values
    for x in df_teams["team_id"]:
        team_values[x] = 0
    for x in player_values.keys():
        team_values[
            df_players[df_players["player_id"] == x]["team_id"].values[0]
        ] += player_values[x]
    game = df_games[df_games["game_id"] == game_id]
    game_date = np.datetime_as_string(game["game_date"].values[0])[:10]
    game_comp_id = game["competition_id"].values[0]
    ce_df = ce.read_by_date(game_date)
    league_mean = np.mean(ce_df[ce_df["league"] == game_comp_id].elo)
    for x in player_values.keys():
        if df_players[df_players["player_id"] == x]["is_starter"].values[0]:
            team_name = df_teams[
                df_teams["team_id"]
                != df_players[df_players["player_id"] == x]["team_id"].values[0]
            ]["team_name"].values[0]
            team_elo = ce_df.loc[team_name].elo
            id_to_xt_impact[x] += player_values[x] * (
                player_values[x]
                / team_values[
                    df_players[df_players["player_id"] == x]["team_id"].values[0]
                ]
            )
            id_to_xt_impact_elo[x][1].append(
                (
                    player_values[x]
                    * (
                        player_values[x]
                        / team_values[
                            df_players[df_players["player_id"] == x]["team_id"].values[
                                0
                            ]
                        ]
                    )
                )
                * (team_elo / league_mean)
            )
            id_to_xt_impact_elo[x][0].append(
                df_players[df_players["player_id"] == x]["minutes_played"].values[0]
            )
            id_to_xt_impact_elo[x][2].append(
                id_to_xt_impact_elo[x][1][-1] / id_to_xt_impact_elo[x][0][-1]
            )

In [ ]:
df_res = pd.DataFrame.from_dict(
    {
        "ids": [x for x in id_to_name.keys()],
        "names": [id_to_name[x] for x in id_to_name.keys()],
        "impact": [id_to_xt_impact[x] for x in id_to_name.keys()],
        "impact_elo": [np.sum(id_to_xt_impact_elo[x][1]) for x in id_to_name.keys()],
    }
)

In [ ]:
df_res.sort_values("impact", ascending=False).head(20)